# BrainWeb: 20 Anatomical Models of 20 Normal Brains

**Download and Preprocessing for PET-MR Simulations**

This notebook will not re-download/re-process files if they already exist.

- Output data
  + `subject_*.npz`: dtype(shape): `float32(127, 344, 344)`
- [Raw data source](http://brainweb.bic.mni.mcgill.ca/brainweb/anatomic_normal_20.html)
  + `subject_*.bin.gz`: dtype(shape): `uint16(362, 434, 362)`
- Prerequisites
  + Python: [requirements.txt](requirements.txt) (e.g. `pip install -r requirements.txt`)

----

- Author: Casper da Costa-Luis <<casper.dcl@physics.org>>
- Date: 2017-19
- Licence: [MPLv2.0](https://www.mozilla.org/MPL/2.0)

In [ ]:
from __future__ import print_function, division
%matplotlib notebook
from utils import volshow
import numpy as np
import gzip
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)

## Download

In [ ]:
LINKS = "04 05 06 18 20 38 41 42 43 44 45 46 47 48 49 50 51 52 53 54"
LINKS = [
    'http://brainweb.bic.mni.mcgill.ca/cgi/brainweb1?do_download_alias=subject' +
    i + '_crisp&format_value=raw_short&zip_value=gnuzip&download_for_real=%5BStart+download%21%5D'
       for i in LINKS.split()]

### Python Version

In [ ]:
import re
from os import path
import requests

with tqdm(LINKS, desc="checking", unit="subject") as progress:
    for url in progress:
        out = re.sub('.*(subject)([0-9]+).*', r'\1_\2.bin.gz', url)
        if path.exists(out):
            continue
        progress.set_postfix(downloading=out)
        d = requests.get(url)
        with open(out, 'wb') as fo:
            fo.write(d.content)

### Bash Version

- Requirements
  + `bash`: `curl`, `sed`

This should not be required since the Python version should run on any OS.

Change the type of the cell below to "code" to run it. 

## Read

In [ ]:
files = ! ls subject_*.bin.gz
files.sort()
data = []
for f in tqdm(files, desc="Uncompressing into memory", unit="subject"):
    with gzip.open(f) as fi:
        data.append(np.frombuffer(fi.read(), dtype=np.uint16).reshape(362,434, 362))

In [ ]:
# show last subject
print(files[-1])
volshow(data[-1]);

## Transform

<div style="visibility: hidden">$\ifcsname bm\endcsname\else\newcommand{\bm}[1]{\mathbf{#1}}\fi$</div>
Convert raw image data:

- Siemens Biograph mMR resolution (~2mm) & dimensions (127, 344, 344)
- PET/T1/T2/uMap intensities
- randomised structure for PET/T1/T2
  + $\bm{\theta} \circ (\bm{1} + \gamma[2G_\sigma(\bm{\rho}) - \bm{1}])$
    * $\bm{\rho} = rand(127, 344, 344) \in [0, 1)$
    * Gaussian smoothing $\sigma = 1$
    * $\gamma = \left\{\matrix{1 & \text{for PET}\\ 0.75 & \text{for MR}}\right.$
    * $\bm{\theta}$ is the PET or MR piecewise constant phantom

In [ ]:
from utils import noise, toPetMmr

sigma = 1
petNoise = 1
t1Noise = t2Noise = 0.75

with tqdm(total=len(files), desc="mMR ground truths", unit="subject") as progress:
    for (f, vol) in zip(files, data):
        out = f.replace('.bin.gz', '.npz')
        if path.exists(out):
            progress.update()
            continue
        progress.set_postfix(creating=out)

        pet, uMap, t1, t2 = toPetMmr(vol)
        pet = noise(pet, petNoise, sigma=sigma)
        t1 = noise(t1, t1Noise, sigma=sigma)
        t2 = noise(t2, t2Noise, sigma=sigma)

        np.savez_compressed(out, PET=pet[:,::-1], uMap=uMap[:,::-1], T1=t1[:,::-1], T2=t2[:,::-1])
        progress.update()

In [ ]:
# show last subject
f = files[-1].replace('.bin.gz', '.npz')
vol = np.load(f)
print(f)
volshow(vol['PET'][:, 100:-100, 100:-100], cmap="hot")
volshow(vol['T1'][:, 100:-100, 100:-100])
volshow(vol['T2'][:, 100:-100, 100:-100])
volshow(vol['uMap'][:, 100:-100, 100:-100], cmap="bone");